<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 10:41:29
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  1.57 L
Current:  1.39 C
-------------------
Today PnL: 26.33 K (0.19%)
Current PnL: -22.55 L (-14.97%)
CY Booked + Current PnL: -11.11 L (-7.38%)
-------------------
Total profit:  1.04 L
Total loss:  -23.59 L
-------------------
Total Booked + Current PnL: 15.87 L (12.83%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.23%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.28 C
Est FTT PnL: 88.93 L (63.96%)
Deployed:  1.24 C
Current:  1.39 C
CAGR/XIRR %: 7.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,60.0,H-MC,12.48,112360.0,11856.0,17247.0,-0.87,11.80,15.35,28.96,79.0,0.69,0.82,49.77,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,36.0,M-SC,2.44,83205.0,-17572.0,17664.0,0.03,-17.44,21.23,0.09,245.0,-0.99,0.61,9.31,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,185020.0,20350.0,22979.0,-2.40,12.36,12.42,26.31,89.0,0.89,1.35,21.33,X40N,NTT,AC
57,RELIANCE,1533.00,-14.41,44.0,H-LC,3.71,214219.0,3853.0,24935.0,0.67,1.83,11.64,13.68,37.0,0.15,1.56,18.66,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195869.0,-4269.0,25150.0,-0.25,-2.13,12.84,10.44,4.0,-0.17,1.42,3.31,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,66.67,76.0,H-SC,1.65,152898.0,-26173.0,91907.0,12.80,-14.62,60.11,36.71,135.0,-0.28,1.11,51.71,XR,NTT,BANKS
26,GREENPANEL,537.00,227.52,56.0,M-SC,1.32,146302.0,-36776.0,115754.0,8.62,-20.09,79.12,43.14,230.0,-0.32,1.06,37.62,XY24,NTT,MISC
64,SHALBY,327.00,1226.95,68.0,M-SC,14.20,180561.0,-582.0,46043.0,5.43,-0.32,25.50,25.10,235.0,-0.01,1.31,44.70,XY24,NTT,HEALTHCARE
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,40.41,80448.0,-33101.0,73151.0,3.64,-29.15,90.93,35.27,268.0,-0.45,0.59,96.90,XR,NTT,HOTELS
52,PGHH,17973.08,-27.37,64.0,H-MC,3.21,210345.0,9525.0,59254.0,2.75,4.74,28.17,34.25,80.0,0.16,1.53,10.19,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-46.81,19.0,H-SC,14.34,80740.0,-21760.0,62364.0,-5.59,-21.23,77.24,39.61,114.0,-0.35,0.59,0.00,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-47.61,24.0,M-SC,1.22,88605.0,-2894.0,81641.0,-4.15,-3.16,92.14,86.07,223.0,-0.04,0.64,28.67,XR,NTT,DURABLES
3,ALKYLAMINE,4546.37,-12.24,28.0,H-SC,10.68,81734.0,-19229.0,113757.0,-3.52,-19.05,139.18,93.63,148.0,-0.17,0.59,21.74,SR,ATH,CHEMICALS
48,LTIM,7230.20,-8.67,39.0,H-LC,1.55,226395.0,-19756.0,98957.0,-2.46,-8.03,43.71,32.18,16.0,-0.20,1.65,25.12,X200,ATH,IT
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,185020.0,20350.0,22979.0,-2.40,12.36,12.42,26.31,89.0,0.89,1.35,21.33,X40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1226.95,68.0,M-SC,14.20,180561.0,-582.0,46043.0,5.43,-0.32,25.5,25.10,235.0,-0.01,1.31,44.70,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-186.36,79.0,M-SC,8.22,148329.0,-891.0,145066.0,2.46,-0.60,97.8,96.62,208.0,-0.01,1.08,60.52,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,11.23,124767.0,-17712.0,37230.0,-0.30,-12.43,29.84,13.70,163.0,-0.48,0.91,47.26,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,36.0,M-SC,2.44,83205.0,-17572.0,17664.0,0.03,-17.44,21.23,0.09,245.0,-0.99,0.61,9.31,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220779.0,-48888.0,85221.0,-0.52,-18.13,38.60,13.47,138.0,-0.57,1.61,11.67,XY24,NTT,PAINTS
66,SIS,528.0,2092.23,42.0,H-SC,4.03,88472.0,-22560.0,46165.0,0.51,-20.32,52.18,21.26,156.0,-0.49,0.64,19.53,OX40N,NTT,MISC
18,CERA,9475.0,-23.39,34.0,H-SC,1.63,139852.0,-36051.0,78079.0,-1.27,-20.49,55.83,23.89,149.0,-0.46,1.02,20.78,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,57.0,M-MC,11.47,230507.0,5545.0,162116.0,-0.46,2.46,70.33,74.53,192.0,0.03,1.68,35.51,XY24,BTT,STEEL
32,HINDZINC,730.22,27.88,64.0,M-LC,3.84,208713.0,3637.0,108927.0,-0.55,1.77,52.19,54.89,52.0,0.03,1.52,26.30,X5K,ATH,METALS
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,185020.0,20350.0,22979.0,-2.40,12.36,12.42,26.31,89.0,0.89,1.35,21.33,X40N,NTT,AC
84,VOLTAS,1530.00,-3.60,41.0,H-MC,2.83,202155.0,10413.0,27352.0,0.56,5.43,13.53,19.69,99.0,0.38,1.47,12.60,XY25,NTT,AC
17,CAMS,5211.76,-7.60,40.0,H-SC,1.83,105165.0,3161.0,40762.0,-1.16,3.10,38.76,43.06,122.0,0.08,0.77,20.75,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,60.0,H-MC,12.48,112360.0,11856.0,17247.0,-0.87,11.80,15.35,28.96,79.0,0.69,0.82,49.77,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,9.38,189328.0,9520.0,102900.0,-0.25,5.29,54.35,62.52,88.0,0.09,1.38,41.19,XR,NTT,BANKS
32,HINDZINC,730.22,27.88,64.0,M-LC,3.84,208713.0,3637.0,108927.0,-0.55,1.77,52.19,54.89,52.0,0.03,1.52,26.30,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.01,25.0,H-SC,9.99,125387.0,2051.0,131707.0,-0.04,1.66,105.04,108.45,119.0,0.02,0.91,24.39,AR,ATH,MISC
86,WIPRO,420.00,-16.06,37.0,M-LC,5.82,147789.0,-3156.0,112615.0,-0.42,-2.09,76.20,72.51,53.0,-0.03,1.08,3.81,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-32.22,27.0,H-LC,12.45,269912.0,-76044.0,150908.0,-0.59,-21.98,55.91,21.64,1.0,-0.50,1.96,0.00,X40,ATH,IT
5,ASIANPAINT,3465.66,-18.68,28.0,H-LC,5.83,205260.0,-46508.0,99715.0,-0.74,-18.47,48.58,21.13,27.0,-0.47,1.49,10.41,X40,ATH,PAINTS
44,JIOFIN,387.00,-6.66,29.0,H-LC,12.86,253571.0,-13794.0,79241.0,0.56,-5.16,31.25,24.48,48.0,-0.17,1.84,46.98,XY24,BTT,FINANCE
83,VBL,671.64,-21.40,32.0,H-LC,9.25,283208.0,-32634.0,145286.0,0.05,-10.33,51.30,35.67,5.0,-0.22,2.06,2.09,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.25,217916.0,-24008.0,87275.0,0.21,-9.92,40.05,26.15,8.0,-0.28,1.59,2.98,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-8.67,39.0,H-LC,1.55,226395.0,-19756.0,98957.0,-2.46,-8.03,43.71,32.18,16.0,-0.20,1.65,25.12,X200,ATH,IT
65,SIEMENS,4671.5,-2.61,45.0,H-LC,1.97,156550.0,-29545.0,77023.0,0.06,-15.88,49.20,25.51,15.0,-0.38,1.14,15.37,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,41.0,H-LC,2.09,195869.0,-4269.0,25150.0,-0.25,-2.13,12.84,10.44,4.0,-0.17,1.42,3.31,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.41,44.0,H-LC,3.71,214219.0,3853.0,24935.0,0.67,1.83,11.64,13.68,37.0,0.15,1.56,18.66,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-12.55,38.0,H-LC,3.74,268960.0,3534.0,53254.0,-0.30,1.33,19.80,21.40,11.0,0.07,1.96,8.76,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-32.22,27.0,H-LC,12.45,269912.0,-76044.0,150908.0,-0.59,-21.98,55.91,21.64,1.0,-0.50,1.96,0.00,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.25,283208.0,-32634.0,145286.0,0.05,-10.33,51.30,35.67,5.0,-0.22,2.06,2.09,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.25,217916.0,-24008.0,87275.0,0.21,-9.92,40.05,26.15,8.0,-0.28,1.59,2.98,X40,ATH,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195869.0,-4269.0,25150.0,-0.25,-2.13,12.84,10.44,4.0,-0.17,1.42,3.31,X40,NTT,FMCG
40,INFY,2275.00,-21.75,36.0,H-LC,8.21,304973.0,-7915.0,179599.0,-0.69,-2.53,58.89,54.87,3.0,-0.04,2.22,3.95,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,45.0,H-LC,1.97,156550.0,-29545.0,77023.0,0.06,-15.88,49.20,25.51,15.0,-0.38,1.14,15.37,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195869.0,-4269.0,25150.0,-0.25,-2.13,12.84,10.44,4.0,-0.17,1.42,3.31,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,28.0,H-LC,5.83,205260.0,-46508.0,99715.0,-0.74,-18.47,48.58,21.13,27.0,-0.47,1.49,10.41,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.87,39.0,H-LC,3.99,205623.0,1123.0,40282.0,0.87,0.55,19.59,20.25,10.0,0.03,1.50,9.33,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.41,44.0,H-LC,3.71,214219.0,3853.0,24935.0,0.67,1.83,11.64,13.68,37.0,0.15,1.56,18.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-32.22,27.0,H-LC,12.45,269912.0,-76044.0,150908.0,-0.59,-21.98,55.91,21.64,1.0,-0.50,1.96,0.00,X40,ATH,IT
40,INFY,2275.00,-21.75,36.0,H-LC,8.21,304973.0,-7915.0,179599.0,-0.69,-2.53,58.89,54.87,3.0,-0.04,2.22,3.95,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.09,195869.0,-4269.0,25150.0,-0.25,-2.13,12.84,10.44,4.0,-0.17,1.42,3.31,X40,NTT,FMCG
83,VBL,671.64,-21.40,32.0,H-LC,9.25,283208.0,-32634.0,145286.0,0.05,-10.33,51.30,35.67,5.0,-0.22,2.06,2.09,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,43.0,H-LC,7.71,248184.0,-13435.0,132654.0,-0.06,-5.14,53.45,45.57,7.0,-0.10,1.81,5.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,40.41,80448.0,-33101.0,73151.0,3.64,-29.15,90.93,35.27,268.0,-0.45,0.59,96.90,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.27,89760.0,-8220.0,29827.0,-1.14,-8.39,33.23,22.05,152.0,-0.28,0.65,30.47,XR,ATH,FINANCE
50,NATIONALUM,244.55,-43.66,60.0,H-MC,12.48,112360.0,11856.0,17247.0,-0.87,11.80,15.35,28.96,79.0,0.69,0.82,49.77,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,11.23,124767.0,-17712.0,37230.0,-0.30,-12.43,29.84,13.70,163.0,-0.48,0.91,47.26,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-186.36,79.0,M-SC,8.22,148329.0,-891.0,145066.0,2.46,-0.60,97.80,96.62,208.0,-0.01,1.08,60.52,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1226.95,68.0,M-SC,14.20,180561.0,-582.0,46043.0,5.43,-0.32,25.50,25.10,235.0,-0.01,1.31,44.70,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-186.36,79.0,M-SC,8.22,148329.0,-891.0,145066.0,2.46,-0.60,97.80,96.62,208.0,-0.01,1.08,60.52,XY25,NTT,FINANCE
69,SURYODAY,240.00,66.67,76.0,H-SC,1.65,152898.0,-26173.0,91907.0,12.80,-14.62,60.11,36.71,135.0,-0.28,1.11,51.71,XR,NTT,BANKS
60,SAIL,228.00,45.95,57.0,M-MC,11.47,230507.0,5545.0,162116.0,-0.46,2.46,70.33,74.53,192.0,0.03,1.68,35.51,XY24,BTT,STEEL
50,NATIONALUM,244.55,-43.66,60.0,H-MC,12.48,112360.0,11856.0,17247.0,-0.87,11.80,15.35,28.96,79.0,0.69,0.82,49.77,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.42
2,50,74.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.13
LC    32.34
MC    23.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.36
X40      14.60
X40N     11.34
XR       11.15
XY25     10.57
AR        8.17
OX40N     7.83
X200      1.65
MH        1.63
X5K       1.52
SR        1.17
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.14
H-LC    25.70
H-MC    20.57
M-SC    13.53
M-LC     5.58
M-MC     2.65
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.93,-7.12,42.47
IT,12.56,-21.87,88.32
FINANCE,10.45,-17.44,70.78
MISC,7.00,-19.32,85.66
BANKS,6.42,-13.05,72.53
PAINTS,5.57,-20.58,38.55
ELECTRICAL,5.44,-10.24,48.92
INSURANCE,3.77,-6.78,46.70
AC,3.53,-0.02,16.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3202597.0,22
XR,1375586.0,14
AR,1210164.0,9
X40,841805.0,10
X40N,648300.0,9
OX40N,571466.0,10
XY25,485897.0,7
SR,259439.0,2
X5K,108927.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3785474.0,29
M-SC,1482407.0,17
H-LC,1288497.0,15
H-MC,1279566.0,15
M-LC,383024.0,4
M-MC,313242.0,2
L-SC,260480.0,3
L-MC,61103.0,1
L-LC,39147.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1244342.0      6
           AR         830003.0      5
M-SC       XY24       816533.0      7
H-SC       XR         796882.0      7
H-MC       XY24       587757.0      4
H-LC       X40        542647.0      5
H-SC       X40N       318362.0      4
M-MC       XY24       313242.0      2
H-SC       OX40N      263891.0      4
           SR         259439.0      2
H-LC       X40N       238725.0      3
H-MC       X40        222611.0      4
H-LC       AR         209677.0      2
H-MC       XY25       182434.0      2
L-SC       XR         171590.0      2
M-SC       AR         170484.0      2
M-LC       XY24       161482.0      2
M-SC       XY25       145066.0      1
           OX40N      143281.0      4
           XR         130496.0      2
H-LC       XY25       119250.0      3
M-LC       XR         112615.0      1
           X5K        108927.0      1
H-MC       XR         102900.0      1
H-LC       X200        98957.0      1
H-MC       X40N        91213.0      2
L-SC       OX40N       88890.0      1
H-LC       XY24        79241.0      1
M-SC       X40         76547.0      1
H-MC       OX40N       75404.0      1
H-SC       MH          72555.0      1
L-MC       XR          61103.0      1
L-LC       XY25        39147.0      1
H-MC       MH          17247.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
